# Index Return data

- using yahoo finance for historical stock return data 
- going to need the following
    - Dates of the announcement
    - Index returns 10 days before the announcement
    - Index returns the day of the announcement
    - Index returns 10 days after the announcement 

In [1]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta


In [20]:
fomc_statements = pd.read_csv('dates/fomc_statements.csv')
fomc_statements['Date'] = pd.to_datetime(fomc_statements['Date'])  


ticker = 'XLRE'  
start_date = fomc_statements['Date'].min() - pd.Timedelta(days=15)  
end_date = fomc_statements['Date'].max() + pd.Timedelta(days=15)  

stock_data = yf.download(ticker, start=start_date, end=end_date)

stock_data['return'] = stock_data['Close'].pct_change()

rows = []

for date in fomc_statements['Date']:
    row = {'announcement_date': date, 'ticker': ticker}  
    for t in range(-10, 11):  
        target_date = date + pd.Timedelta(days=t)
        try:
            row[f'T{t:+}'] = stock_data.loc[target_date, 'return']
        except KeyError:
            row[f'T{t:+}'] = pd.NA      rows.append(row)

final_df = pd.DataFrame(rows)

cols = ['ticker', 'announcement_date'] + [f'T{t:+}' for t in range(-10, 11)]
final_df = final_df[cols]

final_df
# final_df.to_csv('output.csv', index=False) 

/var/folders/hl/fd_cpcmx35x03xwkvsfgw9kc0000gn/T/ipykernel_89003/681803269.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  fomc_statements['Date'] = pd.to_datetime(fomc_statements['Date'])  # ensure it's datetime format
[*********************100%***********************]  1 of 1 completed


,ticker,announcement_date,T-10,T-9,T-8,T-7,T-6,T-5,T-4,T-3,...,T+1,T+2,T+3,T+4,T+5,T+6,T+7,T+8,T+9,T+10
0,XLRE,2000-02-02,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,XLRE,2000-03-21,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,XLRE,2000-05-16,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,XLRE,2000-06-28,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,XLRE,2000-08-22,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,XLRE,2024-09-18,<NA>,"Ticker 0.011918 Name: 2024-09-09 00:00:00,...","Ticker 0.017667 Name: 2024-09-10 00:00:00,...","Ticker -0.002671 Name: 2024-09-11 00:00:00,...","Ticker 0.001785 Name: 2024-09-12 00:00:00,...","Ticker 0.007129 Name: 2024-09-13 00:00:00,...",<NA>,<NA>,...,"Ticker -0.002452 Name: 2024-09-19 00:00:00,...","Ticker -0.001787 Name: 2024-09-20 00:00:00,...",<NA>,<NA>,"Ticker 0.011066 Name: 2024-09-23 00:00:00,...","Ticker 0.0 Name: 2024-09-24 00:00:00, dtyp...","Ticker -0.004235 Name: 2024-09-25 00:00:00,...","Ticker -0.00985 Name: 2024-09-26 00:00:00, ...","Ticker 0.001357 Name: 2024-09-27 00:00:00,...",<NA>
202,XLRE,2024-11-07,"Ticker 0.003177 Name: 2024-10-28 00:00:00,...","Ticker -0.009048 Name: 2024-10-29 00:00:00,...","Ticker 0.003652 Name: 2024-10-30 00:00:00,...","Ticker -0.017512 Name: 2024-10-31 00:00:00,...","Ticker -0.010648 Name: 2024-11-01 00:00:00,...",<NA>,<NA>,"Ticker 0.011231 Name: 2024-11-04 00:00:00,...",...,"Ticker 0.017142 Name: 2024-11-08 00:00:00,...",<NA>,<NA>,"Ticker -0.008654 Name: 2024-11-11 00:00:00,...","Ticker -0.012635 Name: 2024-11-12 00:00:00,...","Ticker 0.007911 Name: 2024-11-13 00:00:00,...","Ticker -0.009003 Name: 2024-11-14 00:00:00,...","Ticker 0.001165 Name: 2024-11-15 00:00:00,...",<NA>,<NA>
203,XLRE,2024-12-18,<NA>,"Ticker 0.002056 Name: 2024-12-09 00:00:00,...","Ticker -0.016188 Name: 2024-12-10 00:00:00,...","Ticker -0.002549 Name: 2024-12-11 00:00:00,...","Ticker -0.000697 Name: 2024-12-12 00:00:00,...","Ticker -0.004185 Name: 2024-12-13 00:00:00,...",<NA>,<NA>,...,"Ticker -0.017174 Name: 2024-12-19 00:00:00,...","Ticker 0.018223 Name: 2024-12-20 00:00:00,...",<NA>,<NA>,"Ticker 0.004559 Name: 2024-12-23 00:00:00,...","Ticker 0.00666 Name: 2024-12-24 00:00:00, ...",<NA>,"Ticker 0.00147 Name: 2024-12-26 00:00:00, ...","Ticker -0.008074 Name: 2024-12-27 00:00:00,...",<NA>
204,XLRE,2025-01-29,<NA>,<NA>,"Ticker 0.018257 Name: 2025-01-21 00:00:00,...","Ticker -0.017452 Name: 2025-01-22 00:00:00,...","Ticker 0.008029 Name: 2025-01-23 00:00:00,...","Ticker 0.002896 Name: 2025-01-24 00:00:00,...",<NA>,<NA>,...,"Ticker 0.01318 Name: 2025-01-30 00:00:00, ...","Ticker -0.002168 Name: 2025-01-31 00:00:00,...",<NA>,<NA>,"Ticker -0.00169 Name: 2025-02-03 00:00:00, ...","Ticker 0.0 Name: 2025-02-04 00:00:00, dtyp...","Ticker 0.015719 Name: 2025-02-05 00:00:00,...","Ticker 0.002857 Name: 2025-02-06 00:00:00,...","Ticker -0.003561 Name: 2025-02-07 00:00:00,...",<NA>


In [10]:

# Load your announcement dates
announcement_dates_df = pd.read_csv("dates/fomc_statements.csv")  # <-- replace with your actual CSV
announcement_dates_df['Date'] = pd.to_datetime(announcement_dates_df['Date'])

# Define the index ticker
ticker = "^GSPC"  # for S&P 500

# Download historical index data
start_date = announcement_dates_df['Date'].min() - timedelta(days=30)
end_date = announcement_dates_df['Date'].max() + timedelta(days=30)
index_data = yf.download(ticker, start=start_date, end=end_date)

# Just use 'Close' price (not 'Adj Close' anymore for indices)
index_data = index_data[['Close']]
index_data = index_data.rename(columns={"Close": "Close_Price"})
index_data = index_data.reset_index()

print(index_data.head())


def get_returns(announcement_date, index_data):
    returns = {}
    
    trading_dates = index_data['Date']
    announcement_date = trading_dates[trading_dates >= announcement_date].iloc[0]

    idx = index_data.index[index_data['Date'] == announcement_date][0]

    if idx >= 10 and idx + 10 < len(index_data):
        price_before = index_data.loc[idx-10, 'Close_Price']
        price_on = index_data.loc[idx, 'Close_Price']
        price_after = index_data.loc[idx+10, 'Close_Price']

        returns['10d_before'] = (price_on - price_before) / price_before
        returns['day_of'] = 0  # will compute later
        returns['10d_after'] = (price_after - price_on) / price_on
    else:
        returns['10d_before'] = None
        returns['day_of'] = None
        returns['10d_after'] = None

    return returns

results = []

for date in announcement_dates_df['Date']:
    returns = get_returns(date, index_data)
    returns['announcement_date'] = date
    results.append(returns)

returns_df = pd.DataFrame(results)

def get_day_of_return(d):
    # Find matching date
    matched = index_data.loc[index_data['Date'] == d, 'Close_Price']
    if not matched.empty:
        return matched.pct_change().iloc[-1]  # Assuming pct_change over that one day
    else:
        import numpy as np
        return np.nan

returns_df['day_of'] = returns_df['announcement_date'].apply(get_day_of_return)


print(returns_df)


/var/folders/hl/fd_cpcmx35x03xwkvsfgw9kc0000gn/T/ipykernel_89003/2384859542.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  announcement_dates_df['Date'] = pd.to_datetime(announcement_dates_df['Date'])
[*********************100%***********************]  1 of 1 completed


Price        Date  Close_Price
Ticker                   ^GSPC
0      2000-01-03  1455.219971
1      2000-01-04  1399.420044
2      2000-01-05  1402.109985
3      2000-01-06  1403.449951
4      2000-01-07  1441.469971


TypeError: object of type 'float' has no len()